# KoChatGPT 업그레이드하기

## 프로젝트 설명

[KoChatGPT](https://github.com/airobotlab/KoChatGPT)의 소스코드를 바탕으로 다양한 모델 개선 전략을 선택해 KoChatGPT를 업그레이드 한다.


## 프로젝트 루브릭

|학습 목표|평가 기준|
|---------|---------|
|기존 KoGPT2와 SFT 적용 모델 결과 분석했는가?|기존 모델의 결과물과 SFT를 적용한 모델의 결과물을 정량/정성적으로 비교/분석했다.|
|SFT 모델과 RM 모델 결과 분석을 해보았는가?|SFT를 적용한 모델의 결과물과 RM을 적용한 모델의 결과물을 정량/정성적으로 비교/분석했다.|
|데이터셋 정제 / 새로운 데이터셋 / foundation model 교체 중 하나를 이용해 정량적 성능 향상을 해보았는가?|1. 기존 데이터셋을 추가로 정제하고, generation 성능을 올리기 위한 기법(Beam search, Top-k sampling 등)을 실험해 모델 성능을 향상시켰다.</br>--------------------------------------------- </br>2. 새로운 데이터를 수집해 전처리를 수행하여 모델의 성능을 향상시켰다.</br>--------------------------------------------- </br>3. 더 적절한 학습 전략(SFT, RM, PPO)을 적용하거나 initial model을 변경해 모델의 성능을 향상시켰다.|

# 환경 세팅

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT

Mounted at /content/drive
/content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT


In [2]:
!sudo apt-get update
!sudo apt-get install -y fonts-nanum fonts-noto-cjk
!fc-list | grep -i "nanum\|noto"
!pip install sentencepiece
!pip install seaborn
!pip install datasets
!pip install loralib
!pip install trl
!pip install accelerate
!pip install transformers

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,196 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/mai

In [3]:
!git clone https://github.com/airobotlab/KoChatGPT
!cp -r KoChatGPT/colossalai_ChatGPT_230319/chatgpt chatgpt

fatal: destination path 'KoChatGPT' already exists and is not an empty directory.


In [4]:
import wandb
import os

os.environ["WANDB_PROJECT"] = "koChatGPT"
wandb.login(key="e879a1d18418199c038fd9543208814ee26e4f29") # 제출할 때는 키 삭제 후 YOUR_API_KEY로 대체
# wandb.login(key="YOUR_API_KEY") # 사용할 때는 weights&bias 회원가입 후 생성되는 API 키 사용

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jylee4747 (jylee4747-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
modifications = [
    {
        "file": "/content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT/chatgpt/trainer/callbacks/save_checkpoint.py",
        "changes": [
            {"line": 3, "old": "from chatgpt.trainer.strategies import ColossalAIStrategy, Strategy",
             "new": "from chatgpt.trainer.strategies import Strategy"},
            {"line": 71, "old": "only_rank0 = not isinstance(self.strategy, ColossalAIStrategy)",
             "new": "            only_rank0 = not isinstance(self.strategy)"},
        ],
    },
    {
        "file": "/content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT/chatgpt/trainer/strategies/__init__.py",
        "changes": [
            {"line": 1, "old": "from .colossalai import ColossalAIStrategy", "new": ""},  # 삭제
            {"line": 5, "old": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy', 'ColossalAIStrategy']",
             "new": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy']"},
        ],
    },
    {
        "file": "/content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT/chatgpt/dataset/reward_dataset.py",
        "changes": [
            {"line": 3, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ],
    },
    {
        "file": "/content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT/chatgpt/trainer/base.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    },
    {
        "file": "/content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT/chatgpt/trainer/rm.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    }
]


def modify_file(file_path, changes):
    """파일에서 지정된 줄을 찾아 내용을 수정하는 함수"""

    if not os.path.exists(file_path):
        print(f"⚠️ 파일이 존재하지 않습니다: {file_path}")
        return

    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    modified = False

    for change in changes:
        line_index = change["line"]
        if 0 <= line_index < len(lines):
            if lines[line_index].strip() == change["old"]:
                lines[line_index] = change["new"] + "\n"
                modified = True
            else:
                print(f"⚠️ {file_path} 파일의 {change['line']}번째 줄이 예상과 다릅니다.")
                print(f"   예상: {change['old']}")
                print(f"   실제: {lines[line_index].strip()}")

    if modified:
        with open(file_path, "w", encoding="utf-8") as file:
            file.writelines(lines)
        print(f"✅ 수정 완료: {file_path}")
    else:
        print(f"⚠️ {file_path} 수정할 내용이 없습니다.")

for mod in modifications:
    modify_file(mod["file"], mod["changes"])

⚠️ /content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT/chatgpt/trainer/callbacks/save_checkpoint.py 파일의 3번째 줄이 예상과 다릅니다.
   예상: from chatgpt.trainer.strategies import ColossalAIStrategy, Strategy
   실제: from chatgpt.trainer.strategies import Strategy
⚠️ /content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT/chatgpt/trainer/callbacks/save_checkpoint.py 파일의 71번째 줄이 예상과 다릅니다.
   예상: only_rank0 = not isinstance(self.strategy, ColossalAIStrategy)
   실제: only_rank0 = not isinstance(self.strategy)
⚠️ /content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT/chatgpt/trainer/callbacks/save_checkpoint.py 수정할 내용이 없습니다.
⚠️ /content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT/chatgpt/trainer/strategies/__init__.py 파일의 1번째 줄이 예상과 다릅니다.
   예상: from .colossalai import ColossalAIStrategy
   실제: 
⚠️ /content/drive/MyDrive/AIFFEL/Deep_Dive/work/KoChatGPT/chatgpt/trainer/strategies/__init__.py 파일의 5번째 줄이 예상과 다릅니다.
   예상: __all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy', 'ColossalAIStrategy']
   실제:

# 라이브러리 호출

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import logging

logging.getLogger("matplotlib.font_manager").setLevel(logging.ERROR)

fontpath = "/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf"
fontprop = fm.FontProperties(fname=fontpath, size=12)
plt.rcParams["font.family"] = fontprop.get_name()

print(f"설정된 폰트: {fontprop.get_name()}")

설정된 폰트: NanumBarunGothic


In [8]:
import os
import re
import urllib.request
import zipfile
import json
import sentencepiece as spm
import pandas as pd
import seaborn as sns
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

from typing import Optional, Dict, Sequence
from dataclasses import dataclass
import logging
import copy
from copy import deepcopy

from tqdm import tqdm
import random

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# 만일 아래 모듈이 불러와지지 않는다면 Clone 및 수정을 잘 진행했는지 확인해주세요.
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer.strategies import NaiveStrategy
from chatgpt.trainer.rm import RewardModelTrainer
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 라이브러리 버전 확인

In [9]:
print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
print("transformers version: {}".format(transformers.__version__)) # transformers 4.28.0
print("GPU 사용 가능여부: {}".format(torch.cuda.is_available()))

Torch version:2.9.0+cu126
Cuda version: 12.6
transformers version: 4.57.2
GPU 사용 가능여부: True


# 사전 정의

In [10]:
# 기본 모델 / 데이터 경로
BASE_MODEL_NAME = "skt/kogpt2-base-v2"
DATA_SFT_PATH = "KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl"
DATA_RM_PATH = "KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl"
DATA_PPO_PATH = "KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl"

# 프롬프트 템플릿 (네 코드 그대로 유지)
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

In [11]:
def create_experiment_dirs(exp_name: str, root_dir: str = "experiments") -> Dict[str, str]:
    """
    실험 이름(exp_name)마다 새로운 폴더를 만들고,
    그 안에 sft / rm / ppo / analysis 서브 폴더를 생성한다.
    """
    exp_dir = os.path.join(root_dir, exp_name)
    if os.path.exists(exp_dir):
        print(f"[경고] {exp_dir} 이미 존재합니다. 같은 이름의 실험을 덮어쓸 수 있습니다.")
    os.makedirs(exp_dir, exist_ok=True)

    sub_dirs = {}
    for sub in ["sft", "rm", "ppo", "analysis"]:
        sub_path = os.path.join(exp_dir, sub)
        os.makedirs(sub_path, exist_ok=True)
        sub_dirs[sub] = sub_path

    print(f"[실험 폴더 생성 완료] {exp_dir}")
    return sub_dirs

# 토크나이저 정의

In [12]:
def load_tokenizer() -> transformers.PreTrainedTokenizer:
    tokenizer = AutoTokenizer.from_pretrained(
        BASE_MODEL_NAME,
        bos_token="</s>",
        eos_token="</s>",
        unk_token="</s>",
        pad_token="</s>",
        padding_side="right",
        model_max_length=512,
    )
    print("[토크나이저 로드 완료]", tokenizer)
    return tokenizer

# SFT 데이터셋 불러오기 및 데이터 정제

In [13]:
def load_sft_raw_data(path: str = DATA_SFT_PATH):
    """
    SFT용 원본 jsonl 데이터를 그대로 로드한다.
    """
    with open(path, "r", encoding="utf-8-sig") as jf:
        data = json.load(jf)
    print(f"[SFT 데이터 로드] {path}, 샘플 수: {len(data)}")
    return data

In [14]:
def clean_sft_data(
    list_data_dict,
    min_completion_len: int = 5,
    max_completion_percentile: float = 95.0,
):
    """
    SFT용 데이터 정제:
    - completion이 너무 짧은 샘플 제거
    - completion 길이 상위 percentile 컷
    - prompt 중복 제거
    """
    lengths = np.array([len(ex["completion"]) for ex in list_data_dict])
    max_len_thr = np.percentile(lengths, max_completion_percentile)

    cleaned = []
    seen_prompts = set()

    for ex in list_data_dict:
        p = ex["prompt"]
        c = ex["completion"]
        if len(c) < min_completion_len:
            continue
        if len(c) > max_len_thr:
            continue
        if p in seen_prompts:
            continue
        seen_prompts.add(p)
        cleaned.append(ex)

    print(
        f"[clean_sft_data] before={len(list_data_dict)}, "
        f"after={len(cleaned)}, removed={len(list_data_dict)-len(cleaned)}"
    )
    return cleaned

In [15]:
class SFT_dataset(Dataset):
    """
    네가 준 SFT_dataset 코드를 최대한 유지하되,
    data_path 대신 리스트를 직접 받을 수 있게 약간만 일반화.
    """

    def __init__(self, data_list, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading SFT data...")

        pattern_instruction = "prompt"   # instruction
        pattern_output = "completion"    # response

        PROMPT_DICT_LOCAL = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT_LOCAL["prompt_input"]

        # 네 코드와 동일: sources, targets, examples 구성
        sources = []
        for example in data_list:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in data_list:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        # source / source+target 토크나이징
        sources_tokenized = self._tokenize_fn(sources, tokenizer)      # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)    # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = [ids.clone() for ids in input_ids]
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        self.input_ids = input_ids
        self.labels = labels
        logging.warning("Loading SFT data done!!: %d", len(self.labels))

    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item()
            for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            input_ids_lens=input_ids_lens,
        )

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [16]:
@dataclass
class DataCollatorForSupervisedDataset(object):
    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple(
            [instance[key] for instance in instances] for key in ("input_ids", "labels")
        )
        input_ids = pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = pad_sequence(labels, batch_first=True, padding_value=-100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

# SFT 학습 함수

In [17]:
def train_sft(
    exp_name: str,
    exp_dirs: Dict[str, str],
    tokenizer: transformers.PreTrainedTokenizer,
    use_clean_data: bool = False,
    num_train_epochs: int = 3,
    batch_size: int = 8,
) -> str:
    """
    - SFT 데이터 로드 + (옵션) 정제
    - wandb run 이름: KoChatGPT_SFT_baseline / KoChatGPT_SFT_clean
    - 결과 저장: exp_dirs["sft"] / (모델, 토크나이저)
    """
    raw_sft = load_sft_raw_data(DATA_SFT_PATH)
    if use_clean_data:
        data_sft = clean_sft_data(raw_sft)
        run_suffix = "clean"
    else:
        data_sft = raw_sft
        run_suffix = "baseline"

    sft_dir = exp_dirs["sft"]

    # Dataset & collator
    train_dataset = SFT_dataset(data_sft, tokenizer)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

    # 모델 로드 (네 코드 그대로)
    model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME).to(device)

    # wandb run 이름 설정
    run_name = f"{exp_name}_SFT_{run_suffix}"
    wandb.init(
        project="koChatGPT",
        name=run_name,
        reinit=True,
    )

    training_args = TrainingArguments(
        output_dir=sft_dir,
        overwrite_output_dir=True,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=5,
        prediction_loss_only=True,
        fp16=True,
        run_name=run_name,
        report_to=["wandb"],
        eval_strategy="no",
        logging_steps=500,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    trainer.train()

    # 모델 & 토크나이저 저장
    model.save_pretrained(sft_dir)
    tokenizer.save_pretrained(sft_dir)

    wandb.finish()

    print(f"[SFT 학습 완료] 저장 경로: {sft_dir}")
    return sft_dir

# Reward Model 학습 함수

In [18]:
class GPTRM_custom(RewardModel):
    """
    네가 준 GPTRM_custom 클래스 그대로 기반으로.
    """

    def __init__(
        self,
        pretrained: Optional[str] = None,
        config: Optional[GPT2Config] = None,
        checkpoint: bool = False,
        lora_rank: int = 0,
        lora_train_bias: str = "none",
        tokenizer=None,
    ) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            if tokenizer is not None:
                model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained

    def save_pretrained(self, dir):
        if getattr(self, "pretrained", None) is not None:
            self.model.save_pretrained(dir)

In [19]:
def prepare_rm_pairwise_data(data_rm_path: str = DATA_RM_PATH):
    """
    네 코드에 있던 ranking → chosen/rejected 변환 로직을 기반으로,
    pairwise 데이터를 생성하고 train/eval로 나눈다.
    """
    with open(data_rm_path, "r", encoding="utf-8-sig") as json_file:
        list_data_dict = json.load(json_file)

    total_data_ranking2chosen = []

    for tmp in list_data_dict:
        prompt = tmp["prompt"]
        ranking = tmp["ranking"]

        for index in range(1, len(ranking)):
            n = ranking[0]
            m = ranking[index]
            data = {
                "prompt": prompt,
                "chosen": tmp[f"completion_{n}"],
                "rejected": tmp[f"completion_{m}"],
            }
            total_data_ranking2chosen.append(data)

    random.seed(230319)
    random.shuffle(total_data_ranking2chosen)

    train_data = total_data_ranking2chosen[:1000]
    eval_data = total_data_ranking2chosen[1000:1200]
    print(
        f"[RM 데이터] 총 pair={len(total_data_ranking2chosen)}, "
        f"train={len(train_data)}, eval={len(eval_data)}"
    )
    return train_data, eval_data

In [20]:
def train_rm(
    exp_name: str,
    exp_dirs: Dict[str, str],
    tokenizer: transformers.PreTrainedTokenizer,
    rm_lr: float = 5e-5,
    max_epochs: int = 3,
) -> str:
    """
    - RM pairwise 데이터 준비
    - wandb run: {exp_name}_RM
    - 결과 저장: exp_dirs["rm"]
    """
    rm_dir = exp_dirs["rm"]
    train_data, eval_data = prepare_rm_pairwise_data(DATA_RM_PATH)

    train_dataset = RewardDataset(train_data, tokenizer, 512)
    eval_dataset = RewardDataset(eval_data, tokenizer, 512)

    with NaiveStrategy().model_init_context():
        model = GPTRM_custom(
            pretrained=BASE_MODEL_NAME,
            lora_rank=0,
            tokenizer=tokenizer,
        ).to(device)

    run_name = f"{exp_name}_RM"
    wandb.init(
        project="koChatGPT",
        name=run_name,
        reinit=True,
    )

    trainer = RewardModelTrainer(
        model=model,
        strategy=NaiveStrategy(),
        optim=optim.Adam(model.parameters(), lr=rm_lr),
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        batch_size=4,
        max_epochs=max_epochs,
    )

    # 주의: RewardModelTrainer 내부에서 wandb를 활용할 수도 있음.
    trainer.fit(use_lora=0)

    model.save_pretrained(rm_dir)
    tokenizer.save_pretrained(rm_dir)

    wandb.finish()

    print(f"[RM 학습 완료] 저장 경로: {rm_dir}")
    return rm_dir

# PPO 학습 함수

In [21]:
def load_ppo_prompts(data_ppo_path: str = DATA_PPO_PATH):
    with open(data_ppo_path, "r", encoding="utf-8-sig") as jf:
        list_data_dict = json.load(jf)
    list_prompt = [tmp["prompt"] for tmp in list_data_dict]
    print(f"[PPO 프롬프트 로드] {len(list_prompt)}개")
    return list_prompt

In [22]:
def train_ppo(
    exp_name: str,
    exp_dirs: Dict[str, str],
    tokenizer: transformers.PreTrainedTokenizer,
    sft_model_dir: str,
    rm_model_dir: str,
    ppo_epochs: int = 3,
) -> str:
    """
    - actor: SFT 모델
    - critic: RM 모델
    - reward_model: RM backbone + value_head
    - wandb run: {exp_name}_PPO
    - 결과 저장: exp_dirs["ppo"]
    """
    ppo_dir = exp_dirs["ppo"]
    list_prompt = load_ppo_prompts(DATA_PPO_PATH)

    with NaiveStrategy().model_init_context():
        actor = GPTActor(pretrained=sft_model_dir, lora_rank=0).to(device)
        critic = GPTCritic(pretrained=rm_model_dir, lora_rank=0).to(device)
        initial_model = deepcopy(actor)

        # RM backbone 재사용
        rm_backbone = GPTRM_custom(
            pretrained=rm_model_dir,
            lora_rank=0,
            tokenizer=tokenizer,
        ).to(device)
        reward_model = RewardModel(
            deepcopy(rm_backbone.model),
            deepcopy(rm_backbone.value_head),
        ).to(device)

    actor_optim = optim.Adam(actor.parameters(), lr=5e-6)
    critic_optim = optim.Adam(critic.parameters(), lr=5e-6)

    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model = (
        NaiveStrategy().prepare(
            (actor, actor_optim),
            (critic, critic_optim),
            reward_model,
            initial_model,
        )
    )

    def tokenize_fn(texts):
        batch = tokenizer(
            texts, return_tensors="pt", max_length=96, padding=True, truncation=True
        )
        return {k: v.to(device) for k, v in batch.items()}

    run_name = f"{exp_name}_PPO"
    wandb.init(
        project="koChatGPT",
        name=run_name,
        reinit=True,
    )

    ppo_trainer = PPOTrainer(
        NaiveStrategy(),
        actor,
        critic,
        reward_model,
        initial_model,
        actor_optim,
        critic_optim,
        max_epochs=ppo_epochs,
        train_batch_size=8,
        tokenizer=tokenize_fn,
        max_length=128,
        do_sample=True,
        temperature=1.0,
        top_k=50,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    ppo_trainer.fit(
        list_prompt,
        num_episodes=10,
        max_timesteps=3,
        update_timesteps=3,
    )

    actor.model.save_pretrained(ppo_dir)
    tokenizer.save_pretrained(ppo_dir)

    wandb.finish()

    print(f"[PPO 학습 완료] 저장 경로: {ppo_dir}")
    return ppo_dir

# 루브릭 1, 2용 비교 함수 + rm inference 함수

In [23]:
def compare_base_vs_sft(
    exp_dirs: Dict[str, str],
    tokenizer: transformers.PreTrainedTokenizer,
    sft_model_dir: str,
    eval_prompts,
    n_show: int = 3,
) -> pd.DataFrame:
    """
    [루브릭 1 평가 기준]
    - 기존 KoGPT2(base)와 SFT 적용 모델의 결과물을
      정량(길이, 토큰 수 등) / 정성(출력 텍스트)적으로 비교하기 위한 CSV 생성.

    저장 형식:
    - prompt
    - base_output, sft_output
    - len_base, len_sft (문자 길이)
    - toklen_base, toklen_sft (토큰 길이)

    추가:
    - n_show 개의 예시를 콘솔에 출력해서, 정성적인 해석이 바로 가능하도록 함.
    """

    analysis_dir = exp_dirs["analysis"]

    base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME).to(device)
    sft_model = AutoModelForCausalLM.from_pretrained(sft_model_dir).to(device)
    base_model.eval()
    sft_model.eval()

    generation_args = dict(
        num_beams=4,
        repetition_penalty=2.0,
        no_repeat_ngram_size=4,
        eos_token_id=375,
        max_new_tokens=64,
        do_sample=True,
        top_k=50,
        early_stopping=True,
    )

    rows = []
    for p in eval_prompts:
        prompt_full = PROMPT_DICT["prompt_input"].format_map({"prompt": p})
        input_ids = tokenizer.encode(prompt_full, return_tensors="pt").to(device)

        with torch.no_grad():
            out_base = base_model.generate(input_ids, **generation_args)
            out_sft = sft_model.generate(input_ids, **generation_args)

        text_base = tokenizer.decode(out_base[0], skip_special_tokens=True)
        text_sft = tokenizer.decode(out_sft[0], skip_special_tokens=True)

        toklen_base = len(tokenizer.encode(text_base))
        toklen_sft = len(tokenizer.encode(text_sft))

        rows.append(
            {
                "prompt": p,
                "base_output": text_base,
                "sft_output": text_sft,
                "len_base": len(text_base),
                "len_sft": len(text_sft),
                "toklen_base": toklen_base,
                "toklen_sft": toklen_sft,
            }
        )

    df = pd.DataFrame(rows)
    csv_path = os.path.join(analysis_dir, "base_vs_sft.csv")
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"base vs SFT 비교 CSV 저장: {csv_path}")
    print("  - 평균 문자 길이(base, sft):", df["len_base"].mean(), df["len_sft"].mean())
    print("  - 평균 토큰 길이(base, sft):", df["toklen_base"].mean(), df["toklen_sft"].mean())

    # ---------- 정성 분석용 예시 출력 ----------
    print("\nBase vs SFT 출력 비교")
    for i, row in df.head(n_show).iterrows():
        print("\n" + "-" * 80)
        print(f"[예시 {i+1}] 프롬프트:")
        print(row["prompt"])
        print("\n[Base 출력]")
        print(row["base_output"])
        print("\n[SFT 출력]")
        print(row["sft_output"])
        print(f"\n(길이) Base: {row['len_base']}, SFT: {row['len_sft']}")
        print(f"(토큰 수) Base: {row['toklen_base']}, SFT: {row['toklen_sft']}")
    print("-" * 80 + "\n")

    return df

In [24]:
def load_rm_for_inference(rm_model_dir: str, tokenizer: transformers.PreTrainedTokenizer):
    """
    RM inference용 모델 로드
    """
    rm_model = GPTRM_custom(pretrained=rm_model_dir, lora_rank=0, tokenizer=tokenizer).to(device)
    rm_model.eval()
    return rm_model

In [25]:
def compare_sft_vs_rm_applied(
    exp_dirs: Dict[str, str],
    tokenizer: transformers.PreTrainedTokenizer,
    sft_model_dir: str,
    rm_model_dir: str,
    eval_prompts,
    num_candidates: int = 4,
    n_show: int = 3,
) -> pd.DataFrame:
    """
    [루브릭 2 평가 기준]

    - "SFT 모델"의 결과:
      → do_sample=False (그리디/빔 서치 기반의 기본 출력)

    - "RM을 적용한 모델"의 결과:
      → SFT 모델이 num_candidates개 응답을 샘플링(do_sample=True)하고,
         그 중 RM score가 가장 높은 응답을 선택한 것.

    즉, 구조적으로는:
      SFT (생성) + RM (선호도 평가) == "RM을 적용한 모델"

    CSV 컬럼:
    - prompt
    - sft_greedy_output
    - rm_selected_output
    - rm_score_sft_greedy
    - rm_score_rm_selected

    추가:
    - n_show 개 예시에 대해 SFT vs RM-selected 텍스트와 점수를 콘솔에 출력.
    """

    analysis_dir = exp_dirs["analysis"]

    sft_model = AutoModelForCausalLM.from_pretrained(sft_model_dir).to(device)
    sft_model.eval()
    rm_model = load_rm_for_inference(rm_model_dir, tokenizer)

    # 1) SFT 기본(greedy/beam) 출력 설정
    greedy_args = dict(
        num_beams=4,
        repetition_penalty=2.0,
        no_repeat_ngram_size=4,
        eos_token_id=375,
        max_new_tokens=64,
        do_sample=False,
        early_stopping=True,
    )

    # 2) 후보 생성용 샘플링 설정
    sample_args = dict(
        num_beams=1,
        repetition_penalty=2.0,
        no_repeat_ngram_size=4,
        eos_token_id=375,
        max_new_tokens=64,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=1.0,
        early_stopping=True,
    )

    def get_rm_score(text: str) -> float:
        with torch.no_grad():
            ids = tokenizer.encode(text, return_tensors="pt").to(device)
            out = rm_model(ids)
        score = out.cpu().numpy()[0]
        return float(score)

    rows = []
    for p in eval_prompts:
        prompt_full = PROMPT_DICT["prompt_input"].format_map({"prompt": p})
        input_ids = tokenizer.encode(prompt_full, return_tensors="pt").to(device)

        # 1) SFT 기본 출력
        with torch.no_grad():
            out_sft_greedy = sft_model.generate(input_ids, **greedy_args)
        text_sft_greedy = tokenizer.decode(out_sft_greedy[0], skip_special_tokens=True)
        rm_sft_greedy = get_rm_score(text_sft_greedy)

        # 2) 후보 N개 생성 → RM이 최고의 후보 선택
        candidate_texts = []
        candidate_scores = []

        with torch.no_grad():
            for _ in range(num_candidates):
                out_cand = sft_model.generate(input_ids, **sample_args)
                text_cand = tokenizer.decode(out_cand[0], skip_special_tokens=True)
                score_cand = get_rm_score(text_cand)
                candidate_texts.append(text_cand)
                candidate_scores.append(score_cand)

        best_idx = int(np.argmax(candidate_scores))
        text_rm_selected = candidate_texts[best_idx]
        rm_rm_selected = candidate_scores[best_idx]

        rows.append(
            {
                "prompt": p,
                "sft_greedy_output": text_sft_greedy,
                "rm_selected_output": text_rm_selected,
                "rm_score_sft_greedy": rm_sft_greedy,
                "rm_score_rm_selected": rm_rm_selected,
            }
        )

    df = pd.DataFrame(rows)
    csv_path = os.path.join(analysis_dir, "sft_vs_rm_applied.csv")
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"SFT vs 'RM 적용 모델' 비교 CSV 저장: {csv_path}")
    print("  - 평균 RM 점수 (SFT greedy):", df["rm_score_sft_greedy"].mean())
    print("  - 평균 RM 점수 (RM-selected):", df["rm_score_rm_selected"].mean())

    # ---------- 정성 분석용 예시 출력 ----------
    print("\nSFT(greedy) vs RM-selected 출력 비교")
    for i, row in df.head(n_show).iterrows():
        print("\n" + "-" * 80)
        print(f"[예시 {i+1}] 프롬프트:")
        print(row["prompt"])
        print("\n[SFT(greedy) 출력]")
        print(row["sft_greedy_output"])
        print(f"(RM 점수: {row['rm_score_sft_greedy']:.3f})")

        print("\n[RM-selected 출력]")
        print(row["rm_selected_output"])
        print(f"(RM 점수: {row['rm_score_rm_selected']:.3f})")
    print("-" * 80 + "\n")

    return df

# 루브릭 3용 간단 정량 요약 함수

In [26]:
def summarize_rubrics(
    exp_name: str,
    use_clean_data: bool,
    df_base_sft: pd.DataFrame,
    df_sft_rm: pd.DataFrame,
) -> Dict[str, float]:
    """
    루브릭 1, 2, 3을 한 실험 안에서 요약하고,
    나중에 baseline vs clean을 비교할 수 있도록 핵심 수치를 리턴한다.

    반환값 예:
    {
        "mean_len_base": ...,
        "mean_len_sft": ...,
        "mean_rm_sft_greedy": ...,
        "mean_rm_rm_selected": ...
    }
    """

    print("\n" + "=" * 60)
    print(f"[실험 요약] {exp_name}")
    print("=" * 60)
    print(f"- 데이터 정제 사용 여부(use_clean_data): {use_clean_data}")

    mean_len_base = df_base_sft["len_base"].mean()
    mean_len_sft = df_base_sft["len_sft"].mean()
    mean_toklen_base = df_base_sft["toklen_base"].mean()
    mean_toklen_sft = df_base_sft["toklen_sft"].mean()

    print("\nBase vs SFT 출력 길이/토큰 비교")
    print(f"  - Base 평균 문자 길이    : {mean_len_base:.1f}")
    print(f"  - SFT  평균 문자 길이    : {mean_len_sft:.1f}")
    print(f"  - Base 평균 토큰 길이    : {mean_toklen_base:.1f}")
    print(f"  - SFT  평균 토큰 길이    : {mean_toklen_sft:.1f}")

    mean_rm_sft_greedy = df_sft_rm["rm_score_sft_greedy"].mean()
    mean_rm_rm_selected = df_sft_rm["rm_score_rm_selected"].mean()

    print("\nSFT vs 'RM 적용 모델' RM 점수 비교")
    print(f"  - SFT(greedy) 평균 RM 점수 : {mean_rm_sft_greedy:.3f}")
    print(f"  - RM-selected 평균 RM 점수 : {mean_rm_rm_selected:.3f}")

    metrics = {
        "mean_len_base": float(mean_len_base),
        "mean_len_sft": float(mean_len_sft),
        "mean_toklen_base": float(mean_toklen_base),
        "mean_toklen_sft": float(mean_toklen_sft),
        "mean_rm_sft_greedy": float(mean_rm_sft_greedy),
        "mean_rm_rm_selected": float(mean_rm_rm_selected),
    }
    return metrics

# 전체 파이프라인 함수

In [27]:
def run_full_pipeline(
    exp_name: str,
    use_clean_data: bool = False,
    num_sft_epochs: int = 3,
    num_rm_epochs: int = 3,
    num_ppo_epochs: int = 3,
):
    """
    전체 파이프라인:
      1) 실험 폴더 생성
      2) 토크나이저 로드
      3) SFT 학습 (wandb)
      4) RM 학습 (wandb)
      5) PPO 학습 (wandb)
      6) 루브릭 1·2 평가 (CSV 저장)
      7) 루브릭 3 요약 출력
    """

    # 1) 실험 폴더
    exp_dirs = create_experiment_dirs(exp_name)

    # 2) 토크나이저
    tokenizer = load_tokenizer()

    # 3) SFT 학습
    sft_model_dir = train_sft(
        exp_name=exp_name,
        exp_dirs=exp_dirs,
        tokenizer=tokenizer,
        use_clean_data=use_clean_data,
        num_train_epochs=num_sft_epochs,
    )

    # 4) RM 학습
    rm_model_dir = train_rm(
        exp_name=exp_name,
        exp_dirs=exp_dirs,
        tokenizer=tokenizer,
        rm_lr=5e-5,
        max_epochs=num_rm_epochs,
    )

    # 5) PPO 학습
    ppo_model_dir = train_ppo(
        exp_name=exp_name,
        exp_dirs=exp_dirs,
        tokenizer=tokenizer,
        sft_model_dir=sft_model_dir,
        rm_model_dir=rm_model_dir,
        ppo_epochs=num_ppo_epochs,
    )

    # 6) 루브릭 1·2 평가용 공통 프롬프트
    eval_prompts = [
        "불고기용 고기 한우에요?",
        "리처드 닉슨이 43대 부통령직을 수행한 년도는?",
        "시카고 오헤어 국제공항은 어디에 있어?",
        "오늘 미세먼지 어때?",
    ]

    df_base_sft = compare_base_vs_sft(
        exp_dirs=exp_dirs,
        tokenizer=tokenizer,
        sft_model_dir=sft_model_dir,
        eval_prompts=eval_prompts,
        n_show = len(eval_prompts),
    )

    df_sft_rm = compare_sft_vs_rm_applied(
        exp_dirs=exp_dirs,
        tokenizer=tokenizer,
        sft_model_dir=sft_model_dir,
        rm_model_dir=rm_model_dir,
        eval_prompts=eval_prompts,
        num_candidates=4,
        n_show = len(eval_prompts),
    )

    # 7) 루브릭 1·2·3 결과 요약
    metrics = summarize_rubrics(
        exp_name=exp_name,
        use_clean_data=use_clean_data,
        df_base_sft=df_base_sft,
        df_sft_rm=df_sft_rm,
    )

    # 필요하다면, 추후 비교를 위해 경로/데이터프레임을 리턴
    return {
        "exp_name": exp_name,
        "exp_dirs": exp_dirs,
        "sft_model_dir": sft_model_dir,
        "rm_model_dir": rm_model_dir,
        "ppo_model_dir": ppo_model_dir,
        "df_base_sft": df_base_sft,
        "df_sft_rm": df_sft_rm,
        "metrics": metrics,
    }

# 실험 진행

In [28]:
# 1) 데이터 정제 없이 baseline 실험
result_baseline = run_full_pipeline(
    exp_name="exp1_baseline",
    use_clean_data=False,
    num_sft_epochs=10,
    num_rm_epochs=10,
    num_ppo_epochs=10,
)

[경고] experiments/exp1_baseline 이미 존재합니다. 같은 이름의 실험을 덮어쓸 수 있습니다.
[실험 폴더 생성 완료] experiments/exp1_baseline


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

[토크나이저 로드 완료] GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<usr>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sys>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("<mask>", rstrip=False, lstrip=False

[SFT 데이터 로드] KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl, 샘플 수: 12000


...

Train epoch [9/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [10/10]:   0%|          | 0/3 [00:00<?, ?it/s]

[PPO 학습 완료] 저장 경로: experiments/exp1_baseline/ppo
base vs SFT 비교 CSV 저장: experiments/exp1_baseline/analysis/base_vs_sft.csv
  - 평균 문자 길이(base, sft): 123.5 204.25
  - 평균 토큰 길이(base, sft): 85.75 95.5

Base vs SFT 출력 비교

--------------------------------------------------------------------------------
[예시 1] 프롬프트:
불고기용 고기 한우에요?

[Base 출력]
### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답): #foodie #delicious #instafood #like4like #소통 #좋아요
#팔로우 #선팔 #맞팔 #먹방 #먹스타그램 #맛스타그램 

[SFT 출력]
### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 직접 식품에 관한 정보를 가지고 있지는 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다.子니다.子: 

(길이) Base: 136, SFT: 214
(토큰 수) Base: 96, SFT: 92

--------------------------------------------------------------------------------
[예시 2] 프롬프트:
리처드 닉슨이 43대 부통령직을 수행한 년도는?

[Base 출력]
### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답): #20180416 #미롱_식단 


[SFT 출력]
### Instruction(명령어):

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


SFT vs 'RM 적용 모델' 비교 CSV 저장: experiments/exp1_baseline/analysis/sft_vs_rm_applied.csv
  - 평균 RM 점수 (SFT greedy): 3.3930980563163757
  - 평균 RM 점수 (RM-selected): 3.498792290687561

SFT(greedy) vs RM-selected 출력 비교

--------------------------------------------------------------------------------
[예시 1] 프롬프트:
불고기용 고기 한우에요?

[SFT(greedy) 출력]
### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 직접적으로 식품에 관한 정보를 가지고 있지는 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 따라서 한우를 사용하는 경우도 있고, 다른 종류의 고기를 사용하는 경우도 있습니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는
(RM 점수: 3.138)

[RM-selected 출력]
### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 AI 어시스턴트이기 때문에 직접 식품에 대한 정보를 가지고 있지는 않습니다. 하지만 일반적으로 불고기는 소스와 함께 먹을 때 대표적인 재료로 사용되는 것으로 알려져 있습니다. 다만, 일부 전통적인 한국 음식에서는 쇠고기를 물에 끓여서 조리하는 경우도 있을 수 있으니 참고하시면 됩니다. this beings s
(RM 점수: 3.340)

--------------------------------------------------------------------------------
[예시 2] 프롬프트:
리처드 닉슨이 43대 부통령직을 수행한 년도는?

[SFT(greed

In [29]:
# 2) 데이터 정제 후 실험
result_clean = run_full_pipeline(
    exp_name="exp1_clean",
    use_clean_data=True,
    num_sft_epochs=10,
    num_rm_epochs=10,
    num_ppo_epochs=10,
)

[경고] experiments/exp1_clean 이미 존재합니다. 같은 이름의 실험을 덮어쓸 수 있습니다.
[실험 폴더 생성 완료] experiments/exp1_clean


[토크나이저 로드 완료] GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<usr>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sys>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("<mask>", rstrip=False, lstrip=False

Step,Training Loss
500,2.987000
1000,2.841800
1500,2.727000
2000,2.244600
2500,2.288700
3000,2.130600
3500,1.796400
4000,1.867400
4500,1.634200
5000,1.465000


train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/grad_norm,▆▅▅▅▄█▄▄█▅▅▅▅▄▅▄▇▄▄▄▁▄▃▂▁▄▂▂
train/learning_rate,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
train/loss,██▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁
total_flos,7677865798272000.0
train/epoch,10
train/global_step,14180
train/grad_norm,5.38872
train/learning_rate,0.0
train/loss,0.4448


[SFT 학습 완료] 저장 경로: experiments/exp1_clean/sft
[RM 데이터] 총 pair=20440, train=1000, eval=200


...

Train epoch [7/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [8/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [9/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [10/10]:   0%|          | 0/3 [00:00<?, ?it/s]

[PPO 학습 완료] 저장 경로: experiments/exp1_clean/ppo
base vs SFT 비교 CSV 저장: experiments/exp1_clean/analysis/base_vs_sft.csv
  - 평균 문자 길이(base, sft): 122.75 212.25
  - 평균 토큰 길이(base, sft): 82.25 96.5

Base vs SFT 출력 비교

--------------------------------------------------------------------------------
[예시 1] 프롬프트:
불고기용 고기 한우에요?

[Base 출력]
### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답): #delicious #foodporn #breakfast #koreanfood #followme #like4like #instadaily #selfie #ootd #outf

[SFT 출력]
### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 직접 고기를 판매하지는 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 한우는 대표적인 고급 육류로 알려져 있기 때문에 한우를 사용했을 때 맛과 품질이 좋을 것으로 예상됩니다. 또한 쇠고기는 소고기의 담백

(길이) Base: 151, SFT: 222
(토큰 수) Base: 96, SFT: 96

--------------------------------------------------------------------------------
[예시 2] 프롬프트:
리처드 닉슨이 43대 부통령직을 수행한 년도는?

[Base 출력]
### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):


[SFT 출력]
### Instruction(명령어):

In [30]:
baseline_metrics = result_baseline["metrics"]
clean_metrics = result_clean["metrics"]

print("=== baseline vs clean 정량 비교 ===")
for k in baseline_metrics.keys():
    print(
        f"{k}: baseline={baseline_metrics[k]:.3f}, clean={clean_metrics[k]:.3f}, "
        f"delta={clean_metrics[k]-baseline_metrics[k]:.3f}"
    )

=== baseline vs clean 정량 비교 ===
mean_len_base: baseline=123.500, clean=122.750, delta=-0.750
mean_len_sft: baseline=204.250, clean=212.250, delta=8.000
mean_toklen_base: baseline=85.750, clean=82.250, delta=-3.500
mean_toklen_sft: baseline=95.500, clean=96.500, delta=1.000
mean_rm_sft_greedy: baseline=3.393, clean=-0.600, delta=-3.993
mean_rm_rm_selected: baseline=3.499, clean=-0.540, delta=-4.039


# 결과

## 1. Base vs SFT 비교
### 1.1. 정량 요약

| 실험       | Base 문자 길이 | SFT 문자 길이 | Base 토큰 | SFT 토큰 |
| -------- | ---------- | --------- | ------- | ------ |
| baseline | 123.5      | 204.3     | 85.8    | 95.5   |
| clean    | 122.8      | 212.3     | 82.3    | 96.5   |

- 두 실험 모두 SFT 출력이 Base보다 훨씬 길고 정보량이 많음
- Base는 해시태그·짧은 문구 중심, SFT는 프롬프트에 맞는 설명 중심 응답 생성

### 1.2. 정성 요약
- Base는 “#food #selfie”처럼 프롬프트와 무관한 자동문구가 반복됨
- SFT는 한우·공항·미세먼지 등 질문 의도에 맞춘 서술형 답변을 제공
- 일부 SFT 답변은 중간에 이상한 문자가 끼거나 길이가 길어지기는 하지만,
전반적으로 Base 대비 응답 품질이 명확히 개선됨

## 2. SFT vs RM-selected 비교
### 2.1. 정량 요약
| 실험       | SFT RM 점수 | RM-selected RM 점수 | 차이     |
| -------- | --------- | ----------------- | ------ |
| baseline | 3.393     | 3.499             | +0.106 |
| clean    | -0.600    | -0.540            | +0.060 |

- 두 실험 모두 RM-selected가 RM 기준으로는 SFT보다 약간 높음
- 하지만 baseline·clean에서 RM 점수 절대값 차이가 커서 실험 간 직접 비교는 불가

### 2.2. 정성 요약
- RM-selected는 의도는 좋지만 사실 오류·문맥 불일치가 자주 발생
- SFT(greedy)는 비교적 간결하고 안정적
- 즉, RM이 선택한 응답이 항상 더 좋은 것은 아니며
RM 학습 데이터·스케일 조정이 필요함

## 3. 데이터 정제 효과 분석
### 3.1. 정량 요약
| 지표        | baseline | clean  | 변화             |
| --------- | -------- | ------ | -------------- |
| SFT loss  | 0.5548   | 0.4448 | **-0.11 (개선)** |
| SFT 문자 길이 | 204.25   | 212.25 | +8.0           |
| SFT 토큰 길이 | 95.5     | 96.5   | +1.0           |

- 데이터 정제 후 SFT loss가 낮아짐 → 학습 안정성 향상
- 응답은 약간 길어졌으나 긍정적인 변화
- 다만 RM 점수는 clean 실험에서 낮게 나타남(스케일 문제)
→ RM 품질 자체를 평가하는 지표로 사용하기엔 부적절

### 3.2. 정성 요약
- 정제 이후 SFT 응답은 문장 구성과 설명 흐름이 약간 더 안정적
- 그러나 RM-selected 품질이 clean에서 더 좋다고 말하기는 어려움
- 데이터 정제는 SFT 단계에서는 효과,
RM/PPO 단계에서는 추가적인 튜닝 필요

## 4. Weights & Bias 시각화
[wandb 대시보드](https://wandb.ai/jylee4747-none/koChatGPT/workspace?nw=nwuserjylee4747)

![result_imgs/results1](result_imgs/results1.png)

SFT 학습 결과 loss의 감소 경향은 데이터 정제 전후로 비슷했으나 데이터 정제 후의 loss가 전체적으로 조금씩 낮게 계산되었고 최종적으로도 0.11 정도 낮았다.

## 5. 전체 결론
- Base 대비 SFT 모델은 프롬프트에 맞는 길고 정보 중심의 답변을 생성하여 명확한 품질 개선이 확인됨.
- RM-selected 응답은 평균 RM 점수는 소폭 높았지만, 정성적으로는 사실 오류와 문맥 이탈이 종종 발생, RM 모델의 개선 필요성이 드러남.
- 데이터 정제는 SFT loss 감소와 응답 안정성 측면에서 긍정적 효과가 있었으며, 루브릭 1·2·3을 모두 충족할 수 있는 결과를 제공함.

# 회고

- 이번에는 SFT 학습 할 때만 Weights&Bias에 자동으로 연동되고 있어 사용했지만 RM이나 PPO도 Weights&Bias에 연동해서 자동으로 시각화 할 수 있도록 수정해 보고 싶다.
- 지난 프로젝트, 이번 프로젝트를 진행하면서 wandb를 사용하는 것에 익숙해 진 것 같아 앞으로도 자주 사용하면 좋을 것 같다.